***
__GEECol classes__

The 'GEECol' daughter classes represent specific ('product') collections of images. The classes contain the information and algorithms needed to retrieve data from gee and calculate, convert... this data into the desired products for the specified region (roi). e.g. the GEECol_s2ndvi class collects the Sentinel 2 NIR and RED data from gee and applies a normalized difference algorithm on this data.

__GEECol.getcollection(...) method__

Collects an ee.ImageCollection, representing a time series of images, over a specified interval, over a specified roi.
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils
import geeproduct


__Proba V 333m__

In [ ]:
#
# GEECol product
#
geecol        = geeproduct.GEECol_pv333ndvi()
#geecol        = geeproduct.GEECol_pv333ndvi_he() # historical vito ndvi scaling [-0.08,0.92] -> [0,250] with 255 as no-data

#
# GEECol.getcollection parameters
#
eepoint       = geeutils.tapspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = geeutils.fleecycloudsday.advance(1, 'week')
roidiameter   = 3
verbose       = False
#
# get collection as-is (not using refcollection and refroipixelsdiameter)
#
geecollection = geecol.getcollection(eedatefrom, eedatetill, eepoint, roidiameter, verbose=verbose)
szdescription = geecollection.get('gee_description').getInfo()
eerefroi      = ee.Geometry(geecollection.get('gee_refroi'))
eecenterpoint = ee.Geometry(geecollection.get('gee_centerpoint'))
eefirstimage  = geecollection.first()
szdate        = eefirstimage.get('gee_date').getInfo()
#
# estimate range in roi environment 
#
xenv = geeutils.squarerasterboundsroi(eecenterpoint, roidiameter*5, eefirstimage)
xmin = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xmax = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
print(szdescription, szdate, "min:", xmin, "max:", xmax)
#
# show it
#
map = geemap.Map(height='400px')
map.centerObject(xenv, 15)
map.addLayer(eefirstimage, {'min':xmin, 'max':xmax, 'opacity':0.9}, szdescription)
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 2),       {'palette':'#ff0000'}, 'eerefroi_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint, 20, 3), {'palette':'#ff0000'}, 'eeroicenterpoint_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eepoint,       20, 3), {'palette':'#00ff00'}, 'eepoint')
map


__Sentinel 1 VV/VH bands - ASCENDING/DESCENDING - sigma0/gamma0__

In [ ]:
#geecol        = geeproduct.GEECol_s1sigma0('VV', 'DESCENDING')
#geecol        = geeproduct.GEECol_s1sigma0('VH', 'DESCENDING')
#geecol        = geeproduct.GEECol_s1sigma0('VV', 'ASCENDING')
#geecol        = geeproduct.GEECol_s1sigma0('VH', 'ASCENDING')
#geecol        = geeproduct.GEECol_s1gamma0('VV', 'DESCENDING')
#geecol        = geeproduct.GEECol_s1gamma0('VH', 'DESCENDING')
#geecol        = geeproduct.GEECol_s1gamma0('VV', 'ASCENDING')
geecol        = geeproduct.GEECol_s1gamma0('VH', 'ASCENDING')

eepoint       = geeutils.tapspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = geeutils.fleecycloudsday.advance(1, 'week')
roidiameter   = 10
verbose       = False
#
#
#
geecollection = geecol.getcollection(eedatefrom, eedatetill, eepoint, roidiameter, verbose=verbose)
szdescription = geecollection.get('gee_description').getInfo()
eerefroi      = ee.Geometry(geecollection.get('gee_refroi'))
eecenterpoint = ee.Geometry(geecollection.get('gee_centerpoint'))
eefirstimage  = geecollection.first()
szdate        = eefirstimage.get('gee_date').getInfo()

xenv = geeutils.squarerasterboundsroi(eecenterpoint, roidiameter*5, eefirstimage)
xmin = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xmax = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

print(szdescription, szdate, "min:", xmin, "max:", xmax)

map = geemap.Map(height='400px')
map.centerObject(xenv, 24)
map.addLayer(eefirstimage, {'min':xmin, 'max':xmax, 'opacity':0.9}, szdescription)
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 1),      {'palette':'#ff0000'}, 'eerefroi_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint, 5, 1), {'palette':'#ff0000'}, 'eeroicenterpoint_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eepoint,       5, 1), {'palette':'#00ff00'}, 'eepoint')
map

__Sentinel 1 VV/VH bands - RVI: 4VH/(VV+VH)__ (VV, VH in natural units - using the "S1_GRD_FLOAT" collection)

In [ ]:
geecol        = geeproduct.GEECol_s1rvi('DESCENDING')
#geecol        = geeproduct.GEECol_s1rvi('ASCENDING')

eepoint       = geeutils.tapspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = geeutils.fleecycloudsday.advance(1, 'week')
roidiameter   = 9
verbose       = False
#
#
#
geecollection = geecol.getcollection(eedatefrom, eedatetill, eepoint, roidiameter, verbose=verbose)
szdescription = geecollection.get('gee_description').getInfo()
eerefroi      = ee.Geometry(geecollection.get('gee_refroi'))
eecenterpoint = ee.Geometry(geecollection.get('gee_centerpoint'))
eefirstimage  = geecollection.first()
szdate        = eefirstimage.get('gee_date').getInfo()

xenv = geeutils.squarerasterboundsroi(eecenterpoint, roidiameter*5, eefirstimage)
xmin = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xmax = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

print(szdescription, szdate, "min:", xmin, "max:", xmax)

map = geemap.Map(height='400px')
map.centerObject(xenv, 24)
map.addLayer(eefirstimage, {'min':xmin, 'max':xmax, 'opacity':0.9}, szdescription)
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 1),      {'palette':'#ff0000'}, 'eerefroi_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint, 5, 1), {'palette':'#ff0000'}, 'eeroicenterpoint_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eepoint,       5, 1), {'palette':'#00ff00'}, 'eepoint')
map

__Sentinel 2 radio bands: ndvi__

In [ ]:
geecol        = geeproduct.GEECol_s2ndvi()
#geecol        = geeproduct.GEECol_s2ndvi_he() # historical vito ndvi scaling [-0.08,0.92] -> [0,250] with 255 as no-data

eepoint       = geeutils.tapspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = geeutils.fleecycloudsday.advance(1, 'week')
roidiameter   = 9
verbose       = False
#
#
#
geecollection = geecol.getcollection(eedatefrom, eedatetill, eepoint, roidiameter, verbose=verbose)
szdescription = geecollection.get('gee_description').getInfo()
eerefroi      = ee.Geometry(geecollection.get('gee_refroi'))
eecenterpoint = ee.Geometry(geecollection.get('gee_centerpoint'))
eefirstimage  = geecollection.first()
szdate        = eefirstimage.get('gee_date').getInfo()

xenv = geeutils.squarerasterboundsroi(eecenterpoint, roidiameter*5, eefirstimage)
xmin = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xmax = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

print(szdescription, szdate, "min:", xmin, "max:", xmax)

map = geemap.Map(height='400px')
map.centerObject(xenv, 24)
map.addLayer(eefirstimage, {'min':xmin, 'max':xmax, 'opacity':0.9}, szdescription)
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 1),      {'palette':'#ff0000'}, 'eerefroi_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint, 5, 1), {'palette':'#ff0000'}, 'eeroicenterpoint_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eepoint,       5, 1), {'palette':'#00ff00'}, 'eepoint')
map

__Sentinel 2 radio bands: fapar__

In [ ]:
geecol        = geeproduct.GEECol_s2fapar()
#geecol        = geeproduct.GEECol_s2fapar_he() # historical vito ndvi scaling [0,1] -> [0,200] with 255 as no-data

eepoint       = geeutils.tapspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = geeutils.fleecycloudsday.advance(1, 'week')
roidiameter   = 12
verbose       = False
#
#
#
geecollection = geecol.getcollection(eedatefrom, eedatetill, eepoint, roidiameter, verbose=verbose)
szdescription = geecollection.get('gee_description').getInfo()
eerefroi      = ee.Geometry(geecollection.get('gee_refroi'))
eecenterpoint = ee.Geometry(geecollection.get('gee_centerpoint'))
eefirstimage  = geecollection.first()
szdate        = eefirstimage.get('gee_date').getInfo()

xenv = geeutils.squarerasterboundsroi(eecenterpoint, roidiameter*5, eefirstimage)
xmin = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xmax = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

print(szdescription, szdate, "min:", xmin, "max:", xmax)

map = geemap.Map(height='400px')
map.centerObject(xenv, 24)
map.addLayer(eefirstimage, {'min':xmin, 'max':xmax, 'opacity':0.9}, szdescription)
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 1),      {'palette':'#ff0000'}, 'eerefroi_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint, 5, 1), {'palette':'#ff0000'}, 'eeroicenterpoint_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eepoint,       5, 1), {'palette':'#00ff00'}, 'eepoint')
map

__Sentinel 2 SCL band__

In [ ]:
geecol        = geeproduct.GEECol_s2scl()

eepoint       = geeutils.bobspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = geeutils.fleecycloudsday.advance(1, 'week')
roidiameter   = 12
verbose       = False
#
#
#
geecollection = geecol.getcollection(eedatefrom, eedatetill, eepoint, roidiameter, verbose=verbose)
szdescription = geecollection.get('gee_description').getInfo()
eerefroi      = ee.Geometry(geecollection.get('gee_refroi'))
eecenterpoint = ee.Geometry(geecollection.get('gee_centerpoint'))
eefirstimage  = geecollection.first()
szdate        = eefirstimage.get('gee_date').getInfo()

xenv = geeutils.squarerasterboundsroi(eecenterpoint, roidiameter*5, eefirstimage)
xmin = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xmax = eefirstimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

print(szdescription, szdate, "min:", xmin, "max:", xmax)

map = geemap.Map(height='400px')
map.centerObject(xenv, 16)
map.addLayer(eefirstimage, geeutils.s2sclvisParams, szdescription)
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 1),       {'palette':'#ff0000'}, 'eerefroi_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint, 10, 2), {'palette':'#ff0000'}, 'eeroicenterpoint_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eepoint,       10, 2), {'palette':'#0000ff'}, 'eepoint')
map


__Sentinel 2 SCL ConvMask band__

In [ ]:
geecol        = geeproduct.GEECol_s2sclconvmask()

eepoint       = geeutils.bobspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = geeutils.fleecycloudsday.advance(1, 'week')
roidiameter   = 64
verbose       = False
#
#
#
geecollection = geecol.getcollection(eedatefrom, eedatetill, eepoint, roidiameter, verbose=verbose)
szdescription = geecollection.get('gee_description').getInfo()
eerefroi      = ee.Geometry(geecollection.get('gee_refroi'))
eecenterpoint = ee.Geometry(geecollection.get('gee_centerpoint'))
eefirstimage  = geecollection.first()
szdate        = eefirstimage.get('gee_date').getInfo()


print(szdescription, szdate, "min:", xmin, "max:", xmax)

#
# make clear sky transparant
#
eefirstimage = eefirstimage.updateMask(eefirstimage.eq(1))
#
# use experimental s2rgb product (using 'TCI' bands) as background
#
eedate       = ee.Date(szdate)
eergbimage   = geeproduct.GEECol_s2rgb().collect(eecenterpoint, eedate, eedate.advance(1, 'day')).first()

map = geemap.Map(height='400px')
map.centerObject(eecenterpoint, 13)
map.addLayer(eergbimage, {'min':1,  'max':255}, 'rgb')
map.addLayer(eefirstimage, {'min':0, 'max':1, 'opacity':0.8}, szdescription)
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 1),       {'palette':'#ff0000'}, 'eerefroi_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint, 10, 2), {'palette':'#ff0000'}, 'eeroicenterpoint_' + str(roidiameter))
map.addLayer(geeutils.outlinegeometryimage(eepoint,       10, 2), {'palette':'#0000ff'}, 'eepoint')
map